# Import Libraries and read data

In [15]:
import ujson as json
import pandas as pd

data_file = 'data/new.json'
records = map(json.loads, open(data_file))
df = pd.DataFrame.from_records(records)

# Data Preprocessing  

In [16]:
neighbor_nodes = []
for col1, row in zip(df["node_id"], df["node"]):
    for d in row:
        for k, v in d.items():
            neighbor_nodes.append({"node_id": col1, "neighbor_nodes": k, "direction": v})
df_1 = pd.DataFrame(neighbor_nodes)

In [17]:
transaction_time = []
transaction_list = []
for col1, col2 in zip(df["transaction_val"], df["transaction_time"]):
    for c1 in col1:
        transaction_list.append(c1)
    for c2 in col2:
        transaction_time.append(c2)
df_1["transaction_val"] = transaction_list
df_1["transaction_time"] = transaction_time

df["FA_1_case_1"] = df["labels"].apply(lambda x: x["FA_1_case_1"])
df["FA_1_case_2"] = df["labels"].apply(lambda x: x["FA_1_case_2"]) 

df_final = pd.merge(df_1, df[["node_id","FA_1_case_1", "FA_1_case_2"]], on="node_id", how="left")

In [18]:
def convert_value(self, unit: str, factor: float) -> None:
    if unit == "EU":
        return factor * 1119.5
    elif unit == "USD":
        return factor * 1182.42
    pass

df_final["transaction_val"] = df_final["transaction_val"] * 1182.42

In [19]:
# filter out every transation that is over 10000
df_final = df_final[df_final["transaction_val"] < 10000]

# filter out single transation
tmp = (df_final.groupby(["node_id","direction"]).count() == 1).neighbor_nodes.reset_index().rename(columns={"neighbor_nodes":"single_trans"})
tmp = tmp.loc[tmp.single_trans == False].drop(["single_trans"], axis=1)

# create the df_mask dataframe
# mask_1_case_1 checks if it's a "out" relation 
# mask_1_case_2 checks if it's a "in" relation

df_mask = pd.merge(tmp,df_final, on = ["node_id","direction"], how="left")
df_mask["mask_1_case_1"] = df_mask["direction"].apply(lambda x: True if x == "out" else False)
df_mask["mask_1_case_2"] = df_mask["direction"].apply(lambda x: True if x == "in" else False)

# filter out all nodes, which has total transaction amount < 10000 

tmp_2 = df_mask.groupby(["node_id","direction"]).sum().transaction_val.reset_index()
tmp_2["threshold_tot_trans_val"] = tmp_2["transaction_val"] > 10000
df_mask = pd.merge(df_mask, tmp_2.drop(["transaction_val"],axis=1), on = ["node_id","direction"], how="left")

df_mask["mask_2_case_1"] = df_mask["mask_1_case_1"] & df_mask["threshold_tot_trans_val"]
df_mask["mask_2_case_2"] = df_mask["mask_1_case_2"] & df_mask["threshold_tot_trans_val"]

df_mask

,node_id,direction,neighbor_nodes,transaction_val,transaction_time,FA_1_case_1,FA_1_case_2,mask_1_case_1,mask_1_case_2,threshold_tot_trans_val,mask_2_case_1,mask_2_case_2
0,0x00192fb10df37c9fb26829eb2cc623cd1bf599e8,in,0x95a5147bd3979d0947603a662c16c5d2eac40e17,59.171761,1628079892,False,False,False,True,False,False,False
1,0x00192fb10df37c9fb26829eb2cc623cd1bf599e8,in,0x95a5147bd3979d0947603a662c16c5d2eac40e17,59.270662,1626948176,False,False,False,True,False,False,False
2,0x00192fb10df37c9fb26829eb2cc623cd1bf599e8,in,0x95a5147bd3979d0947603a662c16c5d2eac40e17,59.135005,1623277117,False,False,False,True,False,False,False
3,0x00192fb10df37c9fb26829eb2cc623cd1bf599e8,in,0x95a5147bd3979d0947603a662c16c5d2eac40e17,59.486128,1622007064,False,False,False,True,False,False,False
4,0x00192fb10df37c9fb26829eb2cc623cd1bf599e8,in,0x95a5147bd3979d0947603a662c16c5d2eac40e17,59.228870,1621102734,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1471,0xffa500b7b592fb29d1b29ca98b27058a6927a856,in,0x886534f47921c98f1e4254ebfd14df8680a04e4d,431.348526,1613572843,False,False,False,True,False,False,False
1472,0xffa500b7b592fb29d1b29ca98b27058a6927a856,in,0x886534f47921c98f1e4254ebfd14df8680a04e4d,0.495338,1613564473,False,False,False,True,False,False,False
1473,0xffa500b7b592fb29d1b29ca98b27058a6927a856,in,0x886534f47921c98f1e4254ebfd14df8680a04e4d,716.882377,1613449696,False,False,False,True,False,False,False
1474,0xffa500b7b592fb29d1b29ca98b27058a6927a856,in,0x886534f47921c98f1e4254ebfd14df8680a04e4d,835.345306,1613374049,False,False,False,True,False,False,False


# FA1 detection functions

In [20]:
def thresholds_FA1_time_jump(df_node, start_index):
    node_flag = False
    end_index = df_node.shape[0]
    time_window_start = df_node['transaction_time'].values[start_index]
    time_window_end =  time_window_start + 172801
    index_latest_trx = df_node['transaction_time'].searchsorted(time_window_end, side = 'left') - 1
    trx_sum = df_node['transaction_val'].iloc[start_index:index_latest_trx + 1].sum()
    # amount threshold
    if trx_sum >= 10000:
        node_flag = True
        return node_flag
    # early-stopping
    elif (index_latest_trx == end_index):
        return node_flag
    else:
        # check for duplicated timestamp to avoid unnecessary looping 
        duplicated_ts = df_node[df_node['transaction_time'] == time_window_start]
        start_index = start_index + len(duplicated_ts)
        if start_index == end_index:
            node_flag = False
        else:
            node_flag = thresholds_FA1_time_jump(df_node, start_index)
    return node_flag

In [21]:
def flagger_FA1_time_jump(df_candidates):
    start_index = 0
    node_flags = {}
    #loop over all candidates
    for node in df_candidates['node_id'].unique().tolist():
        df_node = df_candidates[df_candidates['node_id'] == node].sort_values(by='transaction_time').reset_index(drop=True)
        node_flag = thresholds_FA1_time_jump(df_node, start_index)
        node_flags[node] = node_flag
    return node_flags



# Apply functions on data

In [22]:
case1_candidates = df_mask[df_mask['mask_2_case_1'] == True]
case2_candidates = df_mask[df_mask['mask_2_case_2'] == True]

In [23]:
case1_candidates

,node_id,direction,neighbor_nodes,transaction_val,transaction_time,FA_1_case_1,FA_1_case_2,mask_1_case_1,mask_1_case_2,threshold_tot_trans_val,mask_2_case_1,mask_2_case_2
119,0x20f78bbef7153afd365d696ad86ca57fa1c73aa9,out,0x491f32d87d91ab7f08090e7686ef6591c3b8a48f,591.210000,1630020058,False,False,True,False,True,True,False
120,0x20f78bbef7153afd365d696ad86ca57fa1c73aa9,out,0xe6b5d1b388bd15f849301999016fab0c67e83af6,212.623672,1628876734,False,False,True,False,True,True,False
121,0x20f78bbef7153afd365d696ad86ca57fa1c73aa9,out,0xdf832d6905f69880daa615fd8a632167f595c60e,591.210000,1628876266,False,False,True,False,True,True,False
122,0x20f78bbef7153afd365d696ad86ca57fa1c73aa9,out,0x4ad64983349c49defe8d7a4686202d24b25d0ce8,637.544263,1628876120,False,False,True,False,True,True,False
123,0x20f78bbef7153afd365d696ad86ca57fa1c73aa9,out,0x4ad64983349c49defe8d7a4686202d24b25d0ce8,503.746369,1628872877,False,False,True,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1413,0xf837d6f3175f9df73772fba356f31479e67c1735,out,0xae4c57e1f480ee6cee4612aba39021550e637571,126.113388,1631822098,False,False,True,False,True,True,False
1414,0xf837d6f3175f9df73772fba356f31479e67c1735,out,0xe341ab6ac136c5e531406cfdd2c8cdefb2ae8bbb,103.001736,1631822098,False,False,True,False,True,True,False
1415,0xf837d6f3175f9df73772fba356f31479e67c1735,out,0x65dd984ff5fc038b509a6805a51c0656d2aabdd2,787.324875,1631822098,False,False,True,False,True,True,False
1416,0xf837d6f3175f9df73772fba356f31479e67c1735,out,0x1940b01b5119ddf400259997ab1e03e141a0b128,355.343199,1631822012,False,False,True,False,True,True,False


In [24]:
case1_flags = flagger_FA1_time_jump(case1_candidates)
case2_flags = flagger_FA1_time_jump(case2_candidates)

In [25]:
case1_flags

{'0x20f78bbef7153afd365d696ad86ca57fa1c73aa9': False,
 '0x4740fa6b32c5b41ebbf631fe1af41e6fff6e2388': False,
 '0x6390fd17e63e1362d3a9f18eac3b2acf576deb19': True,
 '0x886534f47921c98f1e4254ebfd14df8680a04e4d': False,
 '0x8fac7eb5a860e489ba015b7039c7e404fc192aed': True,
 '0xb779e70f34c5fcf56d3b3864e095e1add3b55b36': False,
 '0xd561de53e344a945e0a7a96da27bdbe627ed8e03': False,
 '0xf837d6f3175f9df73772fba356f31479e67c1735': True}

In [26]:
case2_flags

{'0x28c6c06298d514db089934071355e5743bf21d60': False,
 '0xfa52274dd61e1643d2205169732f29114bc240b3': False}

In [27]:
dummy_flags = [False] * df_final.shape[0]
df_final['Case1_Flag'] = dummy_flags
df_final['Case2_Flag'] = dummy_flags
for node in df_final['node_id']:
    #mask = (int(df_final['node_id']) == node)
    #print(node)
    if node in case1_flags:
        df_final.loc[(df_final['node_id'] == node, 'Case1_Flag')] = case1_flags[node]
    if node in case2_flags:
        df_final.loc[(df_final['node_id'] == node, 'Case2_Flag')] = case2_flags[node]

In [28]:
# Do we have to filter out transaction_val == 0.00

# Testing

## FA1_Case1: 
### - "0x20f78bbef7153afd365d696ad86ca57fa1c73aa9": False, amount threshold does not fulfill
### - "0x4740fa6b32c5b41ebbf631fe1af41e6fff6e2388": False, amount threshold does not fulfill
### - '0x6390fd17e63e1362d3a9f18eac3b2acf576deb19': True, amount and time threshold fulfill at index 2
### - '0x886534f47921c98f1e4254ebfd14df8680a04e4d': False, amount does not fulfill 
### - '0x8fac7eb5a860e489ba015b7039c7e404fc192aed': True, amount and time threshold fulfill at index 3
### - '0xb779e70f34c5fcf56d3b3864e095e1add3b55b36': False, amount does not fulfill
### - '0xd561de53e344a945e0a7a96da27bdbe627ed8e03': False, amount does not fulfill
### - '0xf837d6f3175f9df73772fba356f31479e67c1735': True, amount and time threshold fulfill at index 0 -> only case with duplicated timestamp

## FA1_Case2:
### - '0x28c6c06298d514db089934071355e5743bf21d60': False, amount does not fulfill
### - '0xfa52274dd61e1643d2205169732f29114bc240b3': False, amount does not fulfill

In [29]:
def thresholds_FA1_time_jump(df_node, start_index):
    node_flag = False
    end_index = df_node.shape[0]
    time_window_start = df_node['transaction_time'].values[start_index]
    print("____________________________start_index: ", start_index)
    time_window_end =  time_window_start + 172801
    index_latest_trx = df_node['transaction_time'].searchsorted(time_window_end, side = 'left') - 1
    #print("index_latest_trx: ", index_latest_trx )
    print(df_node['transaction_val'].iloc[start_index:index_latest_trx + 1])
    df_48_hours = df_node['transaction_val'].iloc[start_index:index_latest_trx + 1]
    trx_sum = df_48_hours.sum()
    print("total_amount: ", trx_sum)
    if trx_sum >= 10000:
        print("flag: true")
        node_flag = True
        return node_flag
    # early-stopping 
    elif index_latest_trx == end_index:
        print("early-stopping")
        return node_flag
    else:
        print("continue...")
        duplicated_ts = df_node[df_node['transaction_time'] == time_window_start]
        start_index = start_index + len(duplicated_ts) 
        #len(df_48_hours) - 1
        #len(duplicated_ts)
        if start_index == end_index:
            node_flag = False
        else:
            node_flag = thresholds_FA1_time_jump(df_node, start_index)
    return node_flag

# Case 1

### Node_id == "0x20f78bbef7153afd365d696ad86ca57fa1c73aa9"

In [30]:
 df_test = df_final[df_final["node_id"] == "0x20f78bbef7153afd365d696ad86ca57fa1c73aa9"]
 df_test = df_test.sort_values(["transaction_time"])

In [31]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
1371    34.803849
1370    59.029256
Name: transaction_val, dtype: float64
total_amount:  93.83310490657576
continue...
____________________________start_index:  1
1370    59.029256
Name: transaction_val, dtype: float64
total_amount:  59.0292560322
continue...
____________________________start_index:  2
1392    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  3
1391    2364.84
Name: transaction_val, dtype: float64
total_amount:  2364.84
continue...
____________________________start_index:  4
1390    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  5
1369    2622.139886
Name: transaction_val, dtype: float64
total_amount:  2622.1398862560145
continue...
____________________________start_index:  6
1389    1182.42
Name: transaction_val, dtype: float64
total_amount:  1182.42
continue...
____________________________star

False

### Node_id == "0x4740fa6b32c5b41ebbf631fe1af41e6fff6e2388"

In [32]:
 df_test = df_final[df_final["node_id"] == "0x4740fa6b32c5b41ebbf631fe1af41e6fff6e2388"]
 df_test = df_test.sort_values(["transaction_time"])

In [33]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
484    7737.057043
Name: transaction_val, dtype: float64
total_amount:  7737.0570430974
continue...
____________________________start_index:  1
481    1917.322195
Name: transaction_val, dtype: float64
total_amount:  1917.3221952444
continue...
____________________________start_index:  2
477    1175.219062
Name: transaction_val, dtype: float64
total_amount:  1175.2190622
continue...
____________________________start_index:  3
474    2725.4781
473    4729.6800
Name: transaction_val, dtype: float64
total_amount:  7455.158100000001
continue...
____________________________start_index:  4
473    4729.68
Name: transaction_val, dtype: float64
total_amount:  4729.68
continue...
____________________________start_index:  5
446    8045.435041
Name: transaction_val, dtype: float64
total_amount:  8045.435040553801
continue...
____________________________start_index:  6
696    0.0
700    0.0
699    0.0
698    0.0
702    0.0
701    0.0
697    0.0
694    0.0


False

### Node_id == "0x6390fd17e63e1362d3a9f18eac3b2acf576deb19"

In [34]:
 df_test = df_final[df_final["node_id"] == "0x6390fd17e63e1362d3a9f18eac3b2acf576deb19"]
 df_test = df_test.sort_values(["transaction_time"])


In [35]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
195    184.089175
242    182.579296
Name: transaction_val, dtype: float64
total_amount:  366.6684706618608
continue...
____________________________start_index:  1
242    182.579296
Name: transaction_val, dtype: float64
total_amount:  182.5792957754208
continue...
____________________________start_index:  2
194    9459.360000
241    9434.741253
193     238.982028
240     220.138628
192     146.530126
239     144.037848
Name: transaction_val, dtype: float64
total_amount:  19643.789883008678
flag: true


True

### Node_id == "0x886534f47921c98f1e4254ebfd14df8680a04e4d"

In [36]:
df_test = df_final[df_final["node_id"] == "0x886534f47921c98f1e4254ebfd14df8680a04e4d"]
df_test = df_test.sort_values(["transaction_time"])

In [37]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
339    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  1
338      0.000000
323    223.685178
324     12.092716
Name: transaction_val, dtype: float64
total_amount:  235.7778942486
continue...
____________________________start_index:  2
323    223.685178
324     12.092716
Name: transaction_val, dtype: float64
total_amount:  235.7778942486
continue...
____________________________start_index:  3
324    12.092716
Name: transaction_val, dtype: float64
total_amount:  12.0927157578
continue...
____________________________start_index:  4
337    0.0
336    0.0
335    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  5
336    0.0
335    0.0
334    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  6
335    0.0
334    0.0
Name: transaction_val, dtype: float64
to

False

### Node_id == "0x8fac7eb5a860e489ba015b7039c7e404fc192aed"

In [38]:
df_test = df_final[df_final["node_id"] == "0x8fac7eb5a860e489ba015b7039c7e404fc192aed"]
df_test = df_test.sort_values(["transaction_time"])

In [39]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
1792    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  1
1791    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  2
1790    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  3
1789       0.000000
1785    5084.406000
1788    5090.828599
Name: transaction_val, dtype: float64
total_amount:  10175.23459919322
flag: true


True

### Node_id == "0xb779e70f34c5fcf56d3b3864e095e1add3b55b36"

In [40]:
df_test = df_final[df_final["node_id"] == "0xb779e70f34c5fcf56d3b3864e095e1add3b55b36"]
df_test = df_test.sort_values(["transaction_time"])

In [41]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
725    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  1
726       0.003902
723    7094.520000
722       0.000000
721       0.000000
720       0.000000
Name: transaction_val, dtype: float64
total_amount:  7094.523901986
continue...
____________________________start_index:  2
723    7094.52
722       0.00
721       0.00
720       0.00
Name: transaction_val, dtype: float64
total_amount:  7094.52
continue...
____________________________start_index:  3
722    0.0
721    0.0
720    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  4
721    0.0
720    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  5
720    0.0
Name: transaction_val, dtype: float64
total_amount:  0.0
continue...
____________________________start_index:  6
718    0.0
719    0.0
Name: tra

False

### Node_id == "0xd561de53e344a945e0a7a96da27bdbe627ed8e03"

In [42]:
df_test = df_final[df_final["node_id"] == "0xd561de53e344a945e0a7a96da27bdbe627ed8e03"]
df_test = df_test.sort_values(["transaction_time"])

In [43]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
107     0.0000
108    17.7363
Name: transaction_val, dtype: float64
total_amount:  17.7363
continue...
____________________________start_index:  1
108    17.7363
Name: transaction_val, dtype: float64
total_amount:  17.7363
continue...
____________________________start_index:  2
106       0.000000
105       0.000000
104       0.000000
103       0.000000
102       0.000000
101       0.000000
25      963.672300
100    1021.487981
Name: transaction_val, dtype: float64
total_amount:  1985.1602808861117
continue...
____________________________start_index:  3
105       0.000000
104       0.000000
103       0.000000
102       0.000000
101       0.000000
25      963.672300
100    1021.487981
Name: transaction_val, dtype: float64
total_amount:  1985.1602808861117
continue...
____________________________start_index:  4
104       0.000000
103       0.000000
102       0.000000
101       0.000000
25      963.672300
100    1021.487981
Name: transaction_val,

False

### Node_id == "0xf837d6f3175f9df73772fba356f31479e67c1735"

In [44]:
df_test = df_final[df_final["node_id"] == "0xf837d6f3175f9df73772fba356f31479e67c1735"]
df_test = df_test.sort_values(["transaction_time"])

In [45]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
1625      9.485349
1624    355.343199
1611    694.676290
1612     67.305779
1613    120.541266
           ...    
1414    421.465701
1413    104.010378
1412    577.077701
1411    102.785060
1410    104.109701
Name: transaction_val, Length: 216, dtype: float64
total_amount:  68119.79836257313
flag: true


True

# Case 2

### Node_id == "0x28c6c06298d514db089934071355e5743bf21d60"

In [46]:
df_test = df_final[df_final["node_id"] == "0x28c6c06298d514db089934071355e5743bf21d60"]
df_test = df_test.sort_values(["transaction_time"])

In [47]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
1665    3659.49429
Name: transaction_val, dtype: float64
total_amount:  3659.4942895188
continue...
____________________________start_index:  1
1662    1190.384523
1663    5090.828599
Name: transaction_val, dtype: float64
total_amount:  6281.21312254566
continue...
____________________________start_index:  2
1663    5090.828599
Name: transaction_val, dtype: float64
total_amount:  5090.82859919322
continue...
____________________________start_index:  3
1667    1210.157983
Name: transaction_val, dtype: float64
total_amount:  1210.1579825647977
continue...
____________________________start_index:  4
1664    1058.979634
Name: transaction_val, dtype: float64
total_amount:  1058.9796335914998
continue...
____________________________start_index:  5
1666    29.271055
Name: transaction_val, dtype: float64
total_amount:  29.271055408200002
continue...


False

### Node_id == "0xfa52274dd61e1643d2205169732f29114bc240b3"

In [48]:
df_test = df_final[df_final["node_id"] == "0xfa52274dd61e1643d2205169732f29114bc240b3"]
df_test = df_test.sort_values(["transaction_time"])
df_test

,node_id,neighbor_nodes,direction,transaction_val,transaction_time,FA_1_case_1,FA_1_case_2,Case1_Flag,Case2_Flag
1963,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,182.579296,1609573360,False,False,False,False
1962,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,9434.741253,1609746284,False,False,False,False
1961,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,220.138628,1609747924,False,False,False,False
1960,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,144.037848,1609836579,False,False,False,False
1959,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,209.800124,1610006695,False,False,False,False
1958,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,219.287988,1610178729,False,False,False,False
1957,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,175.126111,1610351827,False,False,False,False
1956,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,129.059863,1610445133,False,False,False,False
1955,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,171.008716,1610610292,False,False,False,False
1954,0xfa52274dd61e1643d2205169732f29114bc240b3,0x6390fd17e63e1362d3a9f18eac3b2acf576deb19,in,180.586202,1610784836,False,False,False,False


In [49]:
thresholds_FA1_time_jump(df_test, 0)

____________________________start_index:  0
1963    182.579296
Name: transaction_val, dtype: float64
total_amount:  182.5792957754208
continue...
____________________________start_index:  1
1962    9434.741253
1961     220.138628
1960     144.037848
Name: transaction_val, dtype: float64
total_amount:  9798.917729003795
continue...
____________________________start_index:  2
1961    220.138628
1960    144.037848
Name: transaction_val, dtype: float64
total_amount:  364.17647588733
continue...
____________________________start_index:  3
1960    144.037848
1959    209.800124
Name: transaction_val, dtype: float64
total_amount:  353.83797227895036
continue...
____________________________start_index:  4
1959    209.800124
1958    219.287988
Name: transaction_val, dtype: float64
total_amount:  429.0881119247962
continue...
____________________________start_index:  5
1958    219.287988
Name: transaction_val, dtype: float64
total_amount:  219.28798759037582
continue...
__________________________

False